In [1]:
from datasets import load_dataset

# Loading the main split of the dataset
ds_main = load_dataset("openai/gsm8k", "main")

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

root_dir = "output"
method_name = "standard"
benchmark = "gsm8k"

model = "gpt-3.5-turbo"
eval_model = "gpt-4o-mini"
seed = 42
num_retries = 1
warming = [0.0]

In [3]:
from agential.core.llm import LLM
import os
llm = LLM(
        model, 
        organization=os.getenv("OPENAI_ORGANIZATION"), 
        temperature=0,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        seed=seed
)
eval_llm = LLM(
    eval_model,
    organization=os.getenv("OPENAI_ORGANIZATION"),
    temperature=0,
    top_p=1,
    frequency_penalty=0.0,
    presence_penalty=0.0,
    seed=seed
)

In [4]:
import wandb
wandb.login()
from agential.prompting.standard.prompting import Standard
method = Standard(
        llm=llm,
        benchmark=benchmark,
)
run = wandb.init(
    project=benchmark, 
    entity="agential",
    config={
        "model": model,
        "eval_model": eval_model,
        "seed": seed,
        "num_retries": num_retries,
        "warming": warming,
    },
    group=method_name,
    tags=[f"method={method_name}", f"model={model}", f"eval_model={eval_model}", f"seed={seed}", f"num_retries={num_retries}", f"warming={warming}"],
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vincenttu. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: vincenttu (agential). Use `wandb login --relogin` to force relogin


In [5]:
ds_main['test'][0]

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}

In [6]:
question = ds_main['test'][0]["question"]
answer = ds_main['test'][0]["answer"]


question , answer

("Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18')

In [7]:
# Inference.
out = method.generate(
    question=question,
    key=answer,
    num_retries=num_retries,
    warming=warming
)

In [8]:
out

StandardOutput(answer='\n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 4\neggs_sold = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\nearnings_per_egg = 2\ntotal_earnings = eggs_sold * earnings_per_egg\nanswer = total_earnings\n```\n', total_prompt_tokens=814, total_completion_tokens=78, total_tokens=892, total_prompt_cost=0.00040699999999999997, total_completion_cost=0.000117, total_cost=0.0005239999999999999, total_prompt_time=2.1793971061706543, total_time=2.1804587841033936, additional_info=[[StandardStepOutput(answer='\n```python\neggs_laid_per_day = 16\neggs_for_breakfast = 3\neggs_for_muffins = 4\neggs_sold = eggs_laid_per_day - eggs_for_breakfast - eggs_for_muffins\nearnings_per_egg = 2\ntotal_earnings = eggs_sold * earnings_per_egg\nanswer = total_earnings\n```\n', answer_response=Response(input_text="Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?